In [1]:
# %load_ext autotime
# %load_ext autoreload
# %autoreload 2

In [1]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")

/scratch/project/open-32-14/pimenol1/miniconda3/envs/esmfold/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True


In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "GPU-22000b49-8fdf-7f72-5433-90aa052d88b4"
# DEVICE_NAME = "cpu" if os.environ["CUDA_VISIBLE_DEVICES"] == "" else "cuda"

In [2]:
torch.cuda.device_count()

1

In [3]:
from pathlib import Path
import os
import tempfile
import warnings
import math
import pandas as pd
import tqdm
from Bio.PDB import PDBParser
from Bio.Data import IUPACData

import torch
import esm
import biotite.structure.io as bsio

from proteinttt.models.esmfold import ESMFoldTTT, DEFAULT_ESMFOLD_TTT_CFG

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 10000)
pd.set_option("max_colwidth", 100)

SABDAB_SUMMARY_PATH = Path("/scratch/project/open-32-14/pimenol1/ProteinTTT/ProteinTTT/data/20250714_0035730_summary.tsv")
OUTPUT_PATH = Path('/scratch/project/open-32-14/pimenol1/ProteinTTT/ProteinTTT/data/predicted_structures')
PDB_FILES_PATH = Path("/scratch/project/open-32-14/pimenol1/ProteinTTT/ProteinTTT/data/20250714_0035730_PDB")
os.makedirs(OUTPUT_PATH, exist_ok=True)

# os.makedirs(f"{OUTPUT_PATH}/esmfold", exist_ok=True)
# os.makedirs(f"{OUTPUT_PATH}/protein_ttt", exist_ok=True)
# os.makedirs(f"{OUTPUT_PATH}/combined", exist_ok=True)

print(f"Summary file: {SABDAB_SUMMARY_PATH}")
print(f"PDB files path: {PDB_FILES_PATH}")
print(f"Output will be saved to: {OUTPUT_PATH}")

Summary file: /scratch/project/open-32-14/pimenol1/ProteinTTT/ProteinTTT/data/20250714_0035730_summary.tsv
PDB files path: /scratch/project/open-32-14/pimenol1/ProteinTTT/ProteinTTT/data/20250714_0035730_PDB
Output will be saved to: /scratch/project/open-32-14/pimenol1/ProteinTTT/ProteinTTT/data/predicted_structures


In [5]:
df = pd.read_csv(SABDAB_SUMMARY_PATH, sep="\t", low_memory=False)
print(f"Entries in summary: {len(df):,}")
df.head()

Entries in summary: 437


,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,compound,organism,heavy_species,light_species,antigen_species,authors,resolution,method,r_free,r_factor,scfv,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid
0,7uow,H,L,0,A,protein,NaN,spike glycoprotein,VIRAL PROTEIN,04/19/23,SARS-Cov2 S protein structure in complex with neutralizing monoclonal antibody 034_32,Homo sapiens; Severe acute respiratory syndrome coronavirus,homo sapiens,homo sapiens,severe acute respiratory syndrome coronavirus,"Patel, A., Ortlund, E.",0.0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV3,IGKV1,Kappa,NaN,NaN,NaN,NaN,NaN
1,7uow,J,I,0,C,protein,NaN,spike glycoprotein,VIRAL PROTEIN,04/19/23,SARS-Cov2 S protein structure in complex with neutralizing monoclonal antibody 034_32,Homo sapiens; Severe acute respiratory syndrome coronavirus,homo sapiens,homo sapiens,severe acute respiratory syndrome coronavirus,"Patel, A., Ortlund, E.",0.0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV3,IGKV1,Kappa,NaN,NaN,NaN,NaN,NaN
2,7xj6,E,D,0,A,protein,NaN,spike glycoprotein,VIRAL PROTEIN/IMMUNE SYSTEM,04/19/23,SARS-CoV-2 BA.1 Spike trimer in complex with 55A8 Fab and 58G6 Fab in the class 1 conformation,Homo sapiens; Severe acute respiratory syndrome coronavirus 2,homo sapiens,homo sapiens,severe acute respiratory syndrome coronavirus2,"Guo, H., Gao, Y., Lu, Y., Yang, H., Ji, X.",0.0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV1,IGKV1,Kappa,NaN,NaN,NaN,NaN,NaN
3,7xj6,G,F,0,B,protein,NaN,spike glycoprotein,VIRAL PROTEIN/IMMUNE SYSTEM,04/19/23,SARS-CoV-2 BA.1 Spike trimer in complex with 55A8 Fab and 58G6 Fab in the class 1 conformation,Homo sapiens; Severe acute respiratory syndrome coronavirus 2,homo sapiens,homo sapiens,severe acute respiratory syndrome coronavirus2,"Guo, H., Gao, Y., Lu, Y., Yang, H., Ji, X.",0.0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV1,IGKV1,Kappa,NaN,NaN,NaN,NaN,NaN
4,7xj6,K,J,0,B,protein,NaN,spike glycoprotein,VIRAL PROTEIN/IMMUNE SYSTEM,04/19/23,SARS-CoV-2 BA.1 Spike trimer in complex with 55A8 Fab and 58G6 Fab in the class 1 conformation,Homo sapiens; Severe acute respiratory syndrome coronavirus 2,homo sapiens,homo sapiens,severe acute respiratory syndrome coronavirus2,"Guo, H., Gao, Y., Lu, Y., Yang, H., Ji, X.",0.0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV1,IGKV3,Kappa,NaN,NaN,NaN,NaN,NaN


In [6]:
def extract_ab_sequences(row: pd.Series, pdb_file: Path):
    """Return (heavy_seq, light_seq) from SAbDab row or PDB."""
    if {"seqH", "seqL"}.issubset(row.index):
        return row.seqH, row.seqL

    from Bio.PDB import PDBParser

    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("ab", pdb_file)
    seq_map = {"H": [], "L": []}
    three_to_one = {
        "ALA": "A", "CYS": "C", "ASP": "D", "GLU": "E", "PHE": "F", "GLY": "G", "HIS": "H",
        "ILE": "I", "LYS": "K", "LEU": "L", "MET": "M", "ASN": "N", "PRO": "P", "GLN": "Q", "ARG": "R",
        "SER": "S", "THR": "T", "VAL": "V", "TRP": "W", "TYR": "Y",
    }
    for chain in structure.get_chains():
        if chain.id in seq_map:
            for res in chain.get_residues():
                if res.id[0] == " ":
                    seq_map[chain.id].append(three_to_one.get(res.resname, "X"))
    return "".join(seq_map["H"]), "".join(seq_map["L"])

for i, row in df.iterrows():
    heavy_seq, light_seq = extract_ab_sequences(row, PDB_FILES_PATH / f"{row.pdb}.pdb")
    df.at[i, "seqH"] = heavy_seq
    df.at[i, "seqL"] = light_seq
    df.at[i, "full_sequence"] = f"{heavy_seq}:{light_seq}"

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

base_model = esm.pretrained.esmfold_v1().eval().to(device)

ttt_cfg = DEFAULT_ESMFOLD_TTT_CFG
ttt_cfg.steps = 10          
ttt_cfg.batch_size = 4           
ttt_cfg.seed = 0          
model = ESMFoldTTT.ttt_from_pretrained(base_model,
                                       ttt_cfg=ttt_cfg,
                                       esmfold_config=base_model.cfg).to(device)

Using cuda


In [ ]:
def predict_structure(model, sequence, pdb_id, tag, out_dir=OUTPUT_PATH):
    with torch.no_grad():
        pdb_str = model.infer_pdb(sequence)

    out_path = out_dir / f"{pdb_id.lower()}_{tag}.pdb"
    out_path.write_text(pdb_str)
    
    struct = bsio.load_structure(out_path, extra_fields=["b_factor"])
    pLDDT = struct.b_factor.mean()
    if pLDDT < 80:
        print(f'pLDDT_{tag}:', pLDDT)
    return pLDDT

In [ ]:
def fold_chain(sequence, pdb_id, *, model, tag, ttt_steps = 10, out_dir = OUTPUT_PATH):
    """
    Predict a structure for `sequence` with TTT
    return pLDDT.
    The PDB is written as <out_dir>/<pdb_id>_<tag>.pdb
    """
    pLDDT_before = predict_structure(model, sequence, pdb_id, tag=f'{tag}_before_ttt', out_dir=out_dir)
    
    if pLDDT_before >= 80:
        # print(f"Skipping TTT for {pdb_id} as pLDDT is already high: {pLDDT_before:.2f}")
        return pLDDT_before, None
    
    model.ttt_cfg.steps = ttt_steps
    model.ttt(sequence)
    
    pLDDT_after = predict_structure(model, sequence, pdb_id, tag=f'{tag}_ttt', out_dir=out_dir)
    
    model.ttt_reset()
    
    return pLDDT_before, pLDDT_after

In [ ]:
seq_cols = ['seqH', 'seqL', 'full_sequence']

if not seq_cols:
    raise ValueError("No sequence columns detected, check your TSV header!")

results = []
for i, row in df.iterrows():
    pdb_id = str(row.get("pdb", f"entry{i}"))

    chain_map = {"H": ["seqh"],
                 "L": ["seql"], "Full": ["full_sequence"]}

    for tag, hints in chain_map.items():
        col = next((c for c in seq_cols if any(h in c.lower() for h in hints)), None)
        if col is None or pd.isna(row[col]):
            continue

        seq = str(row[col]).strip().upper()
        # out_file = OUTPUT_PATH / f"{pdb_id.lower()}_{tag}.pdb"
        # if out_file.exists():           
        #     continue

        try:
            before, after = fold_chain(seq, pdb_id, model=model, tag=tag, ttt_steps=10)
            results.append({"pdb_id": pdb_id,
                            "chain": tag,
                            "length": len(seq),
                            "pLDDT_before" : before,
                            "pLDDT_after": after
                            })
        except Exception as e:
            warnings.warn(f"{pdb_id}{tag}: {e}")

pLDDT_H_before_ttt: 85.10902424242425
Skipping TTT for 7uow as pLDDT is already high: 85.11
pLDDT_L_before_ttt: 87.62647274954072
Skipping TTT for 7uow as pLDDT is already high: 87.63
pLDDT_H_before_ttt: 85.10902424242425
Skipping TTT for 7uow as pLDDT is already high: 85.11
pLDDT_L_before_ttt: 87.62647274954072
Skipping TTT for 7uow as pLDDT is already high: 87.63
pLDDT_H_before_ttt: 89.35503597122303
Skipping TTT for 7xj6 as pLDDT is already high: 89.36


/tmp/ipykernel_213548/2744124344.py:32: UserWarning: 7xj6L: tensors used as indices must be long, byte or bool tensors
  warnings.warn(f"{pdb_id}{tag}: {e}")


pLDDT_H_before_ttt: 89.35503597122303
Skipping TTT for 7xj6 as pLDDT is already high: 89.36
pLDDT_H_before_ttt: 89.35503597122303
Skipping TTT for 7xj6 as pLDDT is already high: 89.36
pLDDT_H_before_ttt: 89.35503597122303
Skipping TTT for 7xj6 as pLDDT is already high: 89.36
pLDDT_H_before_ttt: 89.35503597122303
Skipping TTT for 7xj8 as pLDDT is already high: 89.36


/tmp/ipykernel_213548/2744124344.py:32: UserWarning: 7xj8L: tensors used as indices must be long, byte or bool tensors
  warnings.warn(f"{pdb_id}{tag}: {e}")


pLDDT_H_before_ttt: 89.35503597122303
Skipping TTT for 7xj8 as pLDDT is already high: 89.36
pLDDT_H_before_ttt: 89.35503597122303
Skipping TTT for 7xj8 as pLDDT is already high: 89.36
pLDDT_H_before_ttt: 89.35503597122303
Skipping TTT for 7xj9 as pLDDT is already high: 89.36


...

pLDDT_H_before_ttt: 80.59285565939771
Skipping TTT for 8cw9 as pLDDT is already high: 80.59
pLDDT_L_before_ttt: 84.51823086574655
Skipping TTT for 8cw9 as pLDDT is already high: 84.52
pLDDT_H_before_ttt: 80.59285565939771
Skipping TTT for 8cw9 as pLDDT is already high: 80.59
pLDDT_L_before_ttt: 84.51823086574655
Skipping TTT for 8cw9 as pLDDT is already high: 84.52
pLDDT_H_before_ttt: 80.59285565939771
Skipping TTT for 8cw9 as pLDDT is already high: 80.59
pLDDT_L_before_ttt: 84.51823086574655
Skipping TTT for 8cw9 as pLDDT is already high: 84.52
pLDDT_H_before_ttt: 80.59285565939771
Skipping TTT for 8cw9 as pLDDT is already high: 80.59
pLDDT_L_before_ttt: 84.51823086574655
Skipping TTT for 8cw9 as pLDDT is already high: 84.52
pLDDT_H_before_ttt: 80.59285565939771
Skipping TTT for 8cw9 as pLDDT is already high: 80.59
pLDDT_L_before_ttt: 84.51823086574655
Skipping TTT for 8cw9 as pLDDT is already high: 84.52
pLDDT_H_before_ttt: 80.59285565939771
Skipping TTT for 8cw9 as pLDDT is already 

/tmp/ipykernel_213548/2744124344.py:32: UserWarning: 7qtiL: tensors used as indices must be long, byte or bool tensors
  warnings.warn(f"{pdb_id}{tag}: {e}")


pLDDT_H_before_ttt: 86.36204268292683
Skipping TTT for 7qti as pLDDT is already high: 86.36
pLDDT_H_before_ttt: 86.36204268292683
Skipping TTT for 7qti as pLDDT is already high: 86.36
pLDDT_H_before_ttt: 86.36204268292683
Skipping TTT for 7qti as pLDDT is already high: 86.36


/tmp/ipykernel_213548/2744124344.py:32: UserWarning: 7qtjH: tensors used as indices must be long, byte or bool tensors
  warnings.warn(f"{pdb_id}{tag}: {e}")
/tmp/ipykernel_213548/2744124344.py:32: UserWarning: 7qtjL: tensors used as indices must be long, byte or bool tensors
  warnings.warn(f"{pdb_id}{tag}: {e}")
/tmp/ipykernel_213548/2744124344.py:32: UserWarning: 7qtkH: tensors used as indices must be long, byte or bool tensors
  warnings.warn(f"{pdb_id}{tag}: {e}")
/tmp/ipykernel_213548/2744124344.py:32: UserWarning: 7qtkL: tensors used as indices must be long, byte or bool tensors
  warnings.warn(f"{pdb_id}{tag}: {e}")
/tmp/ipykernel_213548/2744124344.py:32: UserWarning: 7utzH: tensors used as indices must be long, byte or bool tensors
  warnings.warn(f"{pdb_id}{tag}: {e}")
/tmp/ipykernel_213548/2744124344.py:32: UserWarning: 7utzL: tensors used as indices must be long, byte or bool tensors
  warnings.warn(f"{pdb_id}{tag}: {e}")
/tmp/ipykernel_213548/2744124344.py:32: UserWarning:

pLDDT_H_before_ttt: 31.87103883801705
2025-07-22 01:00:00,526 | INFO | step: 0, accumulated_step: 0, loss: None, perplexity: None, ttt_step_time: 0.00000, score_seq_time: 0.00000, eval_step_time: 10.49544, plddt: 31.87104
2025-07-22 01:00:12,144 | INFO | step: 1, accumulated_step: 4, loss: 2.50195, perplexity: None, ttt_step_time: 1.12435, score_seq_time: 0.00000, eval_step_time: 10.49245, plddt: 28.74502
2025-07-22 01:00:24,041 | INFO | step: 2, accumulated_step: 8, loss: 2.13281, perplexity: None, ttt_step_time: 1.12485, score_seq_time: 0.00000, eval_step_time: 10.49235, plddt: 33.24709
2025-07-22 01:00:35,667 | INFO | step: 3, accumulated_step: 12, loss: 2.00195, perplexity: None, ttt_step_time: 1.12673, score_seq_time: 0.00000, eval_step_time: 10.49784, plddt: 30.54404
2025-07-22 01:00:47,291 | INFO | step: 4, accumulated_step: 16, loss: 1.93750, perplexity: None, ttt_step_time: 1.12598, score_seq_time: 0.00000, eval_step_time: 10.49807, plddt: 30.20942
2025-07-22 01:00:59,367 | IN

In [ ]:
results

NameError: name 'results' is not defined

In [ ]:
results = pd.DataFrame(results)
results.to_csv(OUTPUT_PATH / "ttt_results.tsv", sep="\t", index=False)


In [ ]:
# res_df = pd.DataFrame(results)
print(f"Finished {len(results)} chains")
display(results.describe().loc[["count", "mean", "min", "max"]])